# Fairseq train

In [ ]:
!git clone https://github.com/google/sentencepiece

In [ ]:
%cd /content/sentencepiece
%mkdir build
%cd build
!cmake ..
!make -j $(nproc)
!sudo make install
!sudo ldconfig -v

/content/sentencepiece
/content/sentencepiece/build
-- VERSION: 0.1.95
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Looking for pthread_create
-- Looking for pthread_create - not found
-- Looking for pthread_create in pthreads
-- Looking for pthread_create in pthreads - not found
-- Looking for pthread_create in pthread
-- Looking for pthread_create in pthread - found
-- Found Threads: T

In [ ]:
!git clone https://github.com/pytorch/fairseq

Cloning into 'fairseq'...
remote: Enumerating objects: 27783, done.
remote: Counting objects: 100% (204/204), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 27783 (delta 106), reused 143 (delta 89), pack-reused 27579
Receiving objects: 100% (27783/27783), 11.61 MiB | 20.43 MiB/s, done.
Resolving deltas: 100% (20925/20925), done.


In [ ]:
%cd /content/fairseq
!git reset --hard 3e5a2739bce92b2516d060959a1aa3dd51d221dc
!git submodule sync
!git submodule update --init --recursive
!pip install --editable .
!python setup.py install

Submodule 'fairseq/model_parallel/megatron' (https://github.com/ngoyal2707/Megatron-LM) registered for path 'fairseq/model_parallel/megatron'
Submodule 'fairseq/models/huggingface/transformers' (https://github.com/myleott/transformers.git) registered for path 'fairseq/models/huggingface/transformers'
Cloning into '/content/fairseq/fairseq/model_parallel/megatron'...
Cloning into '/content/fairseq/fairseq/models/huggingface/transformers'...
Submodule path 'fairseq/model_parallel/megatron': checked out 'adb23324c222aad0aad89308e70302d996a5eaeb'
Submodule path 'fairseq/models/huggingface/transformers': checked out '839f8a563cefcb7f2048b310024c217e7829a198'
Obtaining file:///content/fairseq
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 61kB 4.8MB/s 
     |████████████████████████████████| 133kB 11.6MB/s 
     |███████████████████

In [ ]:
# !wget https://dl.fbaipublicfiles.com/fairseq/models/mbart/mbart.cc25.v2.tar.gz
!cp /content/drive/MyDrive/mbart.cc25.v2.tar.gz /content/fairseq/
!tar -xzvf mbart.cc25.v2.tar.gz

In [ ]:
!cp /content/drive/MyDrive/Infosystems/RuBART/news-short.csv ./news-short.csv 

In [ ]:
import pandas as pd

df = pd.read_csv('news-short.csv')
df.dropna(inplace=True)

In [ ]:
with open('spm_train_data.txt','w') as f:
    for line in df['text'].tolist():
        f.write(line.replace('\n',' ')+'\n')
    for line in df['title'].tolist():
        f.write(line.replace('\n',' ')+'\n')

In [ ]:
!mkdir preprocessed

In [ ]:
with open('/content/fairseq/preprocessed/train.en_XX','r') as f:
    line = f.readline()

In [ ]:
len(line)

60574

In [ ]:
len(line.split(' '))

8252

In [ ]:
from sklearn.model_selection import train_test_split
X = df['text'].tolist()
y = df['title'].tolist()
x_train,x_val,y_train,y_val = train_test_split(X, y, test_size=0.2, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.5, random_state=42)

file_mapping = {
    'train.en_XX': x_train,
    'train.ru_RU': y_train,
    'valid.en_XX': x_val,
    'valid.ru_RU': y_val,
    'test.en_XX': x_test,
    'test.ru_RU': y_test
}
for k, v in file_mapping.items():
    with open(f'preprocessed/{k}', 'w') as fp:
        fp.writelines(v)

In [ ]:
!spm_train --help

sentencepiece

Usage: spm_train [options] files

   --help (show help)  type: bool default: false
   --version (show version)  type: bool default: false
   --minloglevel (Messages logged at a lower level than this don't actually get logged anywhere)  type: int default: 0
   --input (comma separated list of input sentences)  type: std::string default: ""
   --input_format (Input format. Supported format is `text` or `tsv`.)  type: std::string default: ""
   --model_prefix (output model prefix)  type: std::string default: ""
   --model_type (model algorithm: unigram, bpe, word or char)  type: std::string default: "unigram"
   --vocab_size (vocabulary size)  type: int32 default: 8000
   --accept_language (comma-separated list of languages this model can accept)  type: std::string default: ""
   --self_test_sample_size (the size of self test samples)  type: int32 default: 0
   --character_coverage (character coverage to determine the minimum symbols)  type: double default: 0.9995
   --inpu

In [ ]:
!spm_train --input=/content/fairseq/spm_train_data.txt --model_prefix=bpe --vocab_size=8000 --character_coverage=1.0 --model_type=bpe --input_sentence_size=2000 --hard_vocab_limit=false --train_extremely_large_corpus=true

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: /content/fairseq/spm_train_data.txt
  input_format: 
  model_prefix: bpe
  model_type: BPE
  vocab_size: 8000
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 2000
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 1
  hard_vocab_limit: 0
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
}
normalizer_spec {
  name: nmt_nfkc
  add_dummy_prefix: 1
  remove_extra_whitespaces: 1
  escape_whitespaces: 1
  normal

In [ ]:
%env SPM=/content/sentencepiece/build/src/spm_encode
%env MODEL=spm.model
%env DATA=preprocessed
%env TRAIN=train
%env VALID=valid
%env TEST=test
%env SRC=en_XX
%env TGT=ru_RU

env: SPM=/content/sentencepiece/build/src/spm_encode
env: MODEL=spm.model
env: DATA=preprocessed
env: TRAIN=train
env: VALID=valid
env: TEST=test
env: SRC=en_XX
env: TGT=ru_RU


In [ ]:
!${SPM} --model=${MODEL} < ${DATA}/${TRAIN}.${SRC} > ${DATA}/${TRAIN}.spm.${SRC} &
!${SPM} --model=${MODEL} < ${DATA}/${TRAIN}.${TGT} > ${DATA}/${TRAIN}.spm.${TGT} &
!${SPM} --model=${MODEL} < ${DATA}/${VALID}.${SRC} > ${DATA}/${VALID}.spm.${SRC} &
!${SPM} --model=${MODEL} < ${DATA}/${VALID}.${TGT} > ${DATA}/${VALID}.spm.${TGT} &
!${SPM} --model=${MODEL} < ${DATA}/${TEST}.${SRC} > ${DATA}/${TEST}.spm.${SRC} &
!${SPM} --model=${MODEL} < ${DATA}/${TEST}.${TGT} > ${DATA}/${TEST}.spm.${TGT} &

tcmalloc: large alloc 1440522240 bytes == 0x55d53d348000 @  0x7f3fc6e4c887 0x7f3fc6bb5f61 0x7f3fc6bb7b92 0x7f3fc6bd026a 0x7f3fc6bce2f6 0x55d4f3dd8266 0x55d4f3dd5944 0x7f3fc618abf7 0x55d4f3dd6e5a


In [ ]:
%env DICT=/content/fairseq/mbart.cc25.v2/dict.txt
%env DEST=postprocess
%env NAME=train_data
!fairseq-preprocess \
  --source-lang ${SRC} \
  --target-lang ${TGT} \
  --trainpref ${DATA}/${TRAIN}.bpe \
  --validpref ${DATA}/${VALID}.bpe \
  --testpref ${DATA}/${TEST}.bpe \
  --destdir ${DEST}/${NAME} \
  --thresholdtgt 0 \
  --thresholdsrc 0 \
  --srcdict ${DICT} \
  --tgtdict ${DICT} \
  --workers 70

env: DICT=/content/fairseq/mbart.cc25.v2/dict.txt
env: DEST=postprocess
env: NAME=train_data
2021-05-07 11:47:22 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, bf16=False, bpe=None, checkpoint_shard_count=1, checkpoint_suffix='', cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='postprocess/train_data', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer=None, padding_factor=8, profile=False, quantization_config_path=None, scoring='bleu', seed=1, source_lang='en_XX', srcdict='/content/fairseq/mbart.cc25.v2/dict.txt', target_lang='ru_RU', task='translation', tensorboard_

In [ ]:
%env PRETRAIN=mbart.cc25.v2
%env langs=ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN
%env SAVEDIR=saved_checkpoint
%env MAX_UPDATE=80000
%env WARMUP_UPDATES=500
%env LR=3e-05
%env MAX_TOKENS=605
%env UPDATE_FREQ=2
%env CUDA_VISIBLE_DEVICES=0 
!fairseq-train /content/fairseq/postprocess/train_data/ \
  --encoder-normalize-before --decoder-normalize-before \
  --arch mbart_large --layernorm-embedding \
  --task translation_from_pretrained_bart \
  --source-lang ${SRC} --target-lang ${TGT} \
  --criterion label_smoothed_cross_entropy --label-smoothing 0.2 \
  --optimizer adam --adam-eps 1e-06 --adam-betas '(0.9, 0.98)' \
  --lr-scheduler polynomial_decay --lr 3e-05 --warmup-updates 2500 --total-num-update 40000 \
  --dropout 0.3 --attention-dropout 0.1 --weight-decay 0.0 \
  --max-tokens 1024 --update-freq 2 \
  --skip-invalid-size-inputs-valid-test \
  --save-interval 1 --save-interval-updates 5000 --keep-interval-updates 10 --no-epoch-checkpoints \
  --seed 222 --log-format simple --log-interval 2 \
  --restore-file ${PRETRAIN} \
  --reset-optimizer --reset-meters --reset-dataloader --reset-lr-scheduler \
  --langs ${langs} \
  --ddp-backend c10d\
  --save-dir ${SAVEDIR}

env: PRETRAIN=mbart.cc25.v2
env: langs=ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN
env: SAVEDIR=saved_checkpoint
env: MAX_UPDATE=80000
env: WARMUP_UPDATES=500
env: LR=3e-05
env: MAX_TOKENS=605
env: UPDATE_FREQ=2
env: CUDA_VISIBLE_DEVICES=0
2021-05-07 11:20:52 | INFO | fairseq_cli.train | Namespace(activation_fn='gelu', adam_betas='(0.9, 0.98)', adam_eps=1e-06, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, all_gather_list_size=16384, arch='mbart_large', attention_dropout=0.1, batch_size=None, batch_size_valid=None, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=25.0, cpu=False, criterion='label_smoothed_cross_entropy', cross_self_attention=False, curriculum=0, data='/content/fairseq/postprocess/train_data/', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d'

# Checking and Choosing short data


In [3]:
import pandas as pd

In [4]:
data = pd.read_csv('/content/lenta-ru-news.csv', usecols=['title','text'], skiprows=range(1,6))

data.head()

,title,text
0,Взрыв в центре Москвы: пострадало 30 человек,В зале игровых автоматов в третьем ярусе подзе...
1,Создан департамент ФСБ по борьбе с терроризмом,Указом президента России Бориса Ельцина внесен...
2,Южно-Сахалинск объявлен очагом холеры,Сегодня областной центр Сахалина и Курил получ...
3,Леворадикалы создают предвыборный блок,Бывший шеф Службы безопасности президента Але...
4,Еще одно землетрясение в Турции: один человек ...,"подземный толчок силой 5,2 балла На северо-зап..."


In [8]:
index = []
for i, text in enumerate(data['text']):
    if len(str(text)) < 2000:
        index.append(i)

In [14]:
short_data = data.iloc[a]

491675

In [ ]:
short_data.to_csv("/content/drive/My Drive/RuBART/news-short.csv", index=False)